# Finding the shortest path in an amusement park
Airson Alexis

## Section 1 : Setup and moving functions

In [1]:
# import 
import cv2
import tkinter as tk
from tkinter.simpledialog import askstring
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Function to update the image with last infos
#--------------------------------------------------------------------------------------------------------

def update_display():

    '''
    The aim of this is to update the image size.
    '''

    #--------------------------------------------------------------------------------------------------------
    
    global actual_image # The actual image with the previously added points in red
    global current_zoom, x_offset, y_offset # The zoom parameters
    global zoomed_image
    
    #--------------------------------------------------------------------------------------------------------

    # We zoom on the original image 
    zoomed_image = cv2.resize(actual_image.copy(), None, fx=current_zoom, fy=current_zoom, interpolation=cv2.INTER_LINEAR)

    #--------------------------------------------------------------------------------------------------------
    
    # Calculate the maximum allowable offsets to avoid going out of bounds (the program crahses and return an error in this case)
    max_x_offset = max(zoomed_image.shape[1] - actual_image.shape[1], 0)
    max_y_offset = max(zoomed_image.shape[0] - actual_image.shape[0], 0)

    #--------------------------------------------------------------------------------------------------------
    
    # Limit x_offset and y_offset to stay within bounds
    x_offset = max(0, min(x_offset, max_x_offset))
    y_offset = max(0, min(y_offset, max_y_offset))

    #--------------------------------------------------------------------------------------------------------
    
    # Apply offsets to simulate movement
    zoomed_image = zoomed_image[y_offset:y_offset + actual_image.shape[0], x_offset:x_offset + actual_image.shape[1]]

In [3]:
# Functions to zoom in the image 
#------------------------------------------------------------------------------------------------------------------

def zoom_in():
    
    '''
    This is a function to zoom in the map. 
    '''
    
    global current_zoom, zoom_factor # The zoom parameters
    global zoomed_image
    
    current_zoom *= zoom_factor # Updates the currrent zoom
    
    update_display() # Updates the window of the actual image
    
    # Displays what has been done to this point 
    cv2.imshow('Image', zoomed_image)
    
#------------------------------------------------------------------------------------------------------------------

def zoom_out():
    
    '''
    This is a function to zoom out the map. 
    '''
    
    global current_zoom, zoom_factor # The zoom parameters
    global zoomed_image
    
    current_zoom /= zoom_factor # Updates the currrent zoom
    
    update_display() # Updates the window of the actual image
    
    # Displays what has been done to this point 
    cv2.imshow('Image', zoomed_image)

In [4]:
# Function to move on the zoomed map
#------------------------------------------------------------------------------------------------------------------

def move(direction):
    
    '''
    This is a function to move on the zoomed map.
    '''
    
    global x_offset, y_offset, move_step # The movement parameters
    global zoomed_image
    
    #------------------------------------------------------------------------------------------------------------------
    # To move up on the zoomed map
    if direction == 'up':
        y_offset -= move_step
        
    #------------------------------------------------------------------------------------------------------------------
    # To move down on the zoomed map   
    elif direction == 'down':
        y_offset += move_step
        
    #------------------------------------------------------------------------------------------------------------------
    # To move left on the zoomed map    
    elif direction == 'left':
        x_offset -= move_step
        
    #------------------------------------------------------------------------------------------------------------------
    # To move right on the zoomed map   
    elif direction == 'right':
        x_offset += move_step
        
    #------------------------------------------------------------------------------------------------------------------
        
    update_display()
    
    # Displays what has been done to this point 
    cv2.imshow('Image', zoomed_image)

## Adding points on the map

In [5]:
# Function to add a point
#--------------------------------------------------------------------------------------------------------

def add_point(event, x, y, flags, param):

    '''
    The aim of this function is to add a point on the loaded image. The added point will be a node and should be an interest point on the map.

    Arg :
    event : An action defined in cv2. Here it is the right click to add a point. 
    x,y : The position of the point in the current window (as we allow zoom).
    flags,param : not used, mandatory to use this function
    
    '''
    global actual_image # The actual image with the previously added points in red
    global final # The final image with all ever added points
    
    global all_points # The dictionnary of all ever added points
    
    global x_offset, y_offset, current_zoom # The zoom parameters
    
    global zoomed_image

    # When the right click is pressed 
    if event == cv2.EVENT_RBUTTONDOWN:

        #--------------------------------------------------------------------------------------------------------

        # Store the point in the original image coordinates (as we're working on a zommed window)
        original_x = int((x + x_offset) / current_zoom)
        original_y = int((y + y_offset) / current_zoom)

        #--------------------------------------------------------------------------------------------------------
        
        # Create a Tkinter root window (hidden). Mean window of the program
        root = tk.Tk()
        root.withdraw()

        #--------------------------------------------------------------------------------------------------------
        
        # Display an input dialog to enter the name for the point
        point_name = askstring("Point Name", "Enter a name for the point:")

        #--------------------------------------------------------------------------------------------------------
        
        # Check if a name was entered
        if point_name:
            
            # We add a red circle at pointed point on the actual image
            cv2.circle(actual_image, (original_x, original_y), 5, (0, 0, 255), -1)
            
            # We add a blue circle at pointed point on the final image
            cv2.circle(final, (original_x, original_y), 5, (255, 0, 0), -1)
            
            # Store the point and its coordinates in the dictionaries of the names
            all_points[point_name] = (original_x,original_y)

        #--------------------------------------------------------------------------------------------------------
        
        # We replace the previous version of the images with the new one with our new point
        cv2.imwrite('Map_with_red_points.JPG', actual_image)
        cv2.imwrite('Map_with_blue_points.JPG', final)

        # Destroy the root window
        root.destroy()

        #--------------------------------------------------------------------------------------------------------

        # Update the image using our function defined above
        update_display()
        
        # Displays what has been done to this point 
        cv2.imshow('Image', zoomed_image)

In [6]:
# Mean part of the code to add points 
#------------------------------------------------------------------------------------------------------------------

# We define a dict that will contain all ever added points and their position
all_points = {}

#------------------------------------------------------------------------------------------------------------------

# We load the original image and make a copy of it on which we'll add points 
original_image = cv2.imread('Map.JPG')

# Create a copy of the original that we will update with temporary red points
temporary = original_image.copy()
cv2.imwrite('Map_with_red_points.JPG', temporary)

# Create a copy of the original that we will update with all ever added blue points
final = original_image.copy()
cv2.imwrite('Map_with_blue_points.JPG', final)

#------------------------------------------------------------------------------------------------------------------

boo = True
while boo :  

    #------------------------------------------------------------------------------------------------------------------

    # Zoom parameters
    zoom_factor = 1.2  # You can adjust the zoom factor
    current_zoom = 1.0 # Must be initialized at 1 

    # Movement parameters
    x_offset = 0 # Must be initialized at 0
    y_offset = 0 # Must be initialized at 0
    
    move_step = 30 # You can adjust the movement factor

    #------------------------------------------------------------------------------------------------------------------

    # Load your JPEG image
    actual_image = cv2.imread('Map_with_blue_points.JPG')

    #------------------------------------------------------------------------------------------------------------------

    # Create a window and set the mouse callback function
    cv2.namedWindow('Image')
    cv2.setMouseCallback('Image', add_point)

    #------------------------------------------------------------------------------------------------------------------
    
    # Displays what has been done to this point 
    cv2.imshow('Image', actual_image)

    while True:        

        #------------------------------------------------------------------------------------------------------------------

        # Wait for user to add points and press a key
        key = cv2.waitKey(1) & 0xFF

        if key == ord('p'): # You finished part of the points
            break
        
        elif key == ord('+'): # To zoom in 
            zoom_in()
            
        elif key == ord('-'): # To zoom out
            zoom_out()
            
        elif key == ord('w') : # To move down
            move('down')
            
        elif key ==  ord('z'): # To move up
            move('up')
            
        elif key == ord('q') : # To move left
            move('left')
            
        elif key ==  ord('s'): # To move right
            move('right')

        elif key == ord('y'): # You finished adding all your points
            boo = False 
            break 

    #------------------------------------------------------------------------------------------------------------------

    cv2.destroyAllWindows()

#------------------------------------------------------------------------------------------------------------------

# Print all points with their names
for name, location in all_points.items():
    print(f"Name: {name}, Location: {location}")

#------------------------------------------------------------------------------------------------------------------
# And write them in an external file 

# Specify the name of the output text file
output_file = "points.txt"

# Open the file in write mode to write all names
with open(output_file, "w") as file:
    for name, location in all_points.items():
        file.write(str(name) + ';' + str(location) + '\n')

Name: tiger, Location: (789, 215)
Name: polar_bear, Location: (715, 132)
Name: monkey, Location: (692, 209)
Name: raccoon, Location: (861, 221)
Name: elephants, Location: (652, 74)
Name: white_tiger, Location: (565, 209)
Name: zebras, Location: (485, 135)
Name: hippos, Location: (276, 207)
Name: pandas, Location: (288, 284)
Name: bear, Location: (914, 484)
Name: lion, Location: (389, 172)


In [7]:
# Just to rename the files for more 

if os.path.isfile('Map_with_points.JPG'):
    os.remove('Map_with_points.JPG')
os.rename('Map_with_blue_points.JPG', 'Map_with_points.JPG')
os.remove('Map_with_red_points.JPG')

## Adding paths between the nodes

In [8]:
def calculate_curve_length(curve_points):
    
    '''
    This function computes the "length" of a curve on the map. 
    
    Arguments : 
    points : all the successive points of the curve available
    '''
    
    # Compute the length of the curve using numerical integration (segment wise) 
    length = 0
    
    for i in range(1, len(curve_points)):
        
        x1, y1 = curve_points[i - 1]
        x2, y2 = curve_points[i]
        
        segment_length = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        
        length += segment_length
        
    return length

In [9]:
def endpoints():
    
    '''This function asks the user to select the 2 endpoints of the curve among the nodes'''
    
    #------------------------------------------------------------------------------------------------------------------
    # Extract the names
    
    def read_names_from_file(filename):
        with open(filename, 'r') as file:
            return [line.strip().split(';')[0] for line in file]

    #------------------------------------------------------------------------------------------------------------------
    # Validation of the choices 
    
    def validation():
        # Add the selected names to the 'selected_names' array
        selected_names.append(name_var.get())
        selected_names.append(name_var2.get())
        root.destroy()  # Close the Tkinter window gracefully
        
    #------------------------------------------------------------------------------------------------------------------
    # Cancel 
    
    def cancel():
        root.destroy()  # Close the Tkinter window gracefully
        
    #------------------------------------------------------------------------------------------------------------------

    # Read names from the text file (modify the filename as needed)
    names_list = read_names_from_file('points.txt')

    root = tk.Tk()
    root.title("Choose Endpoints")

    # Create an empty array to store the selected names
    selected_names = []
    
    #------------------------------------------------------------------------------------------------------------------

    # "From" text
    to_label = tk.Label(root, text="From:")
    to_label.pack()
    
    #------------------------------------------------------------------------------------------------------------------

    # Create a placeholder for the first dropdown menu
    name_var = tk.StringVar(root)
    name_var.set("Select a name")  # Placeholder text
    name_dropdown = tk.OptionMenu(root, name_var, *names_list)
    name_dropdown.pack(pady=10)
    
    #------------------------------------------------------------------------------------------------------------------

    # "To" text
    to_label = tk.Label(root, text="To:")
    to_label.pack()
    
    #------------------------------------------------------------------------------------------------------------------

    # Create the second dropdown menu
    name_var2 = tk.StringVar(root)
    name_var2.set("Select a name")  # Placeholder text
    name_dropdown2 = tk.OptionMenu(root, name_var2, *names_list)
    name_dropdown2.pack(pady=10)
    
    #------------------------------------------------------------------------------------------------------------------

    # Create a "validation" button
    validation_button = tk.Button(root, text="Validate", command=validation)
    validation_button.pack(pady=10)
    
    #------------------------------------------------------------------------------------------------------------------
    
    # Create a "cancel" button
    cancel_button = tk.Button(root, text="Cancel", command=cancel)
    cancel_button.pack(pady=10)
    
    #------------------------------------------------------------------------------------------------------------------

    root.mainloop()

    # Now the selected names are stored in the 'selected_names' array
    return selected_names

In [10]:
def draw_curve(event, x, y, flags, param):
    
    '''
    The aim of this function is to add a point on the loaded image. The added point will be a node and should be an interest point on the map.

    Arg :
    event : An action defined in cv2. Here it is the right click to add a point. 
    x,y : The position of the point in the current window (as we allow zoom).
    flags,param : not used, mandatory to use this function
    
    '''
    
    global drawing # A boolean to see if we're holding the button
    global points # An empty list that will contain points of the curves. Must be global
    global actual_image, zoomed_image # The image on which we'll draw red curves
    global count # Numbering the curves allows us to have multiple edges between the same endpoints
    global all_curves, Curves
    global x_offset, y_offset, current_zoom # The zoom parameters
    
    #------------------------------------------------------------------------------------------------------------------

    # We start by clicking on the left button
    if event == cv2.EVENT_LBUTTONDOWN:
        
        # Store the point in the original image coordinates (as we're working on a zommed window)
        original_x = int((x + x_offset) / current_zoom)
        original_y = int((y + y_offset) / current_zoom)
        
        drawing = True
        points = [(original_x, original_y)]

    #------------------------------------------------------------------------------------------------------------------

    # We move the mouse to draw the curve
    elif event == cv2.EVENT_MOUSEMOVE:
        
        # Store the point in the original image coordinates (as we're working on a zommed window)
        original_x = int((x + x_offset) / current_zoom)
        original_y = int((y + y_offset) / current_zoom)
        
        if drawing: # You must hold the left button
            
            points.append((original_x, original_y)) # Keep track of the points among the path
            
            for i in range(1, len(points)):
                
                cv2.line(actual_image, points[i - 1], points[i], (0, 0, 255), 2) # To draw the red curve on the image
                
            cv2.imwrite('Map_with_curves.JPG', actual_image)

    #------------------------------------------------------------------------------------------------------------------
    
    # When we don't hold the button anymore
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

        ends = endpoints() # The endpoints are asked to the user
        
        if len(ends) == 2:
 
            name = (count, ends[0], ends[1]) # Give a name to the curve with its number and the ends
            count += 1
            
            all_curves.append(points)
            Curves[name] = calculate_curve_length(points) # Computes the length and add an entry in the dic

In [11]:
# Mean part of the code to add curves
#------------------------------------------------------------------------------------------------------------------

all_curves = []
Curves = {} 

# To store points along the path
points = []

#------------------------------------------------------------------------------------------------------------------

# Loads the images with points and make a copy on which we'll add curves 
original_image = cv2.imread('Map_with_points.JPG')
actual_image = original_image.copy()
cv2.imwrite('Map_with_curves.JPG', actual_image)

#------------------------------------------------------------------------------------------------------------------

boo = True
drawing = False  # True if mouse button is pressed
count = 1 

#------------------------------------------------------------------------------------------------------------------

# Create a window and set the mouse callback function
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', draw_curve)

#------------------------------------------------------------------------------------------------------------------

boo = True 
while boo:
    
    #------------------------------------------------------------------------------------------------------------------

    # Zoom parameters
    zoom_factor = 1.2  # You can adjust the zoom factor
    current_zoom = 1.0 # Must be initialized at 1 

    # Movement parameters
    x_offset = 0 # Must be initialized at 0
    y_offset = 0 # Must be initialized at 0
    
    move_step = 30 # You can adjust the movement factor

    #------------------------------------------------------------------------------------------------------------------

    # Load your JPEG image
    actual_image = cv2.imread('Map_with_curves.JPG')

    #------------------------------------------------------------------------------------------------------------------
    
    cv2.imshow('Image', actual_image)
    
    while True:
        
        # Wait for user to add points and press a key
        key = cv2.waitKey(1) & 0xFF

        if key == ord('p'): # You finished part of the points
            break
        
        elif key == ord('+'): # To zoom in 
            zoom_in()
            
        elif key == ord('-'): # To zoom out
            zoom_out()
            
        elif key == ord('w') : # To move down
            move('down')
            
        elif key ==  ord('z'): # To move up
            move('up')
            
        elif key == ord('q') : # To move left
            move('left')
            
        elif key ==  ord('s'): # To move right
            move('right')

        elif key == ord('y'): # You finished adding all your points
            boo = False 
            break 

    #------------------------------------------------------------------------------------------------------------------

    # Save the final image with the curves
    for curve in all_curves:
        for i in range(1, len(curve)):
            cv2.line(original_image, curve[i - 1], curve[i], (255, 0, 0), 2)

    cv2.imwrite('Map_with_curves.JPG', original_image)

    #------------------------------------------------------------------------------------------------------------------

cv2.destroyAllWindows()

#------------------------------------------------------------------------------------------------------------------

# Print the edges and their lengths 
for ends, length in Curves.items():
    print(f"Ends: {ends}, Length: {length}")

#------------------------------------------------------------------------------------------------------------------

# Specify the name of the output text file
output_file = "curves.txt"

# Open the file in write mode
with open(output_file, "w") as file:
    for ends, length in Curves.items():
        file.write(str(ends[0]) + ';' + ends[1] + ',' + ends[2] + ';' + str(length) + '\n') # Write the curves in a txt file

Ends: (1, 'raccoon', 'bear'), Length: 373.9163567342129
Ends: (2, 'raccoon', 'bear'), Length: 1078.4036476151962
Ends: (3, 'tiger', 'raccoon'), Length: 71.87171854833812
Ends: (4, 'polar_bear', 'tiger'), Length: 118.60045435298177
Ends: (5, 'monkey', 'polar_bear'), Length: 85.29816560012694
Ends: (6, 'monkey', 'tiger'), Length: 99.2479584390511
Ends: (7, 'elephants', 'polar_bear'), Length: 93.72316996423048
Ends: (8, 'white_tiger', 'monkey'), Length: 129.80582341620806
Ends: (9, 'elephants', 'white_tiger'), Length: 175.45092273846407
Ends: (10, 'zebras', 'white_tiger'), Length: 182.5495558776929
Ends: (11, 'white_tiger', 'zebras'), Length: 114.42932987035393
Ends: (12, 'lion', 'zebras'), Length: 108.71270926514377
Ends: (13, 'lion', 'hippos'), Length: 129.82849130950083
Ends: (14, 'pandas', 'hippos'), Length: 82.28560927016196
Ends: (15, 'lion', 'pandas'), Length: 160.88846102158402
Ends: (16, 'bear', 'white_tiger'), Length: 455.8346637477455
Ends: (17, 'pandas', 'bear'), Length: 684.8

In [ ]:
#last update